In [1]:
import json

# Load both JSON files
with open(r'C:\Users\ahmednabil\Documents\Downloads\excel\processed_data.json', 'r', encoding='utf-8') as f:
    data1 = json.load(f)

with open(r'C:\Users\ahmednabil\Documents\Downloads\excel\formatted_training_data.json', 'r', encoding='utf-8') as f:
    data2 = json.load(f)

# Merge the data
merged_data = data1 + data2

# Save as JSONL
with open(r'C:\Users\ahmednabil\Documents\Downloads\excel\merged_data.jsonl', 'w', encoding='utf-8') as f:
    for item in merged_data:
        json.dump(item, f)
        f.write('\n')


In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load dataset
dataset = load_dataset('json', data_files=r'C:\Users\ahmednabil\Documents\Downloads\excel\merged_data.jsonl', split='train')

# Initialize the model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['prompt'] + examples['completion'], padding="max_length", truncation=True)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["prompt", "completion"])

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")


C:\Users\ahmednabil\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Python311\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
  0%|          | 1/270 [00:21<1:36:18, 21.48s/it]

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the trained model and tokenizer
model_name = "./results"  # Directory where the model is saved
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add a padding token to the tokenizer if not already added
tokenizer.pad_token = tokenizer.eos_token

# Function to generate responses
def generate_response(prompt, max_length=50):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test the function with a sample prompt
prompt = "Q: What is Galala University (GU)?"
response = generate_response(prompt)
print(response)


Q: What is Galala University (GU)?
